# Building a Simple Agent in Python

## Introduction

Welcome to this tutorial on building a simple agent in Python! In this notebook, we will walk through the process of creating a basic agent that can perform tasks autonomously using a model. This is a beginner-friendly tutorial, and while a basic understanding of Python will help, no prior knowledge of agents or state management is required.

By the end of this tutorial, you'll understand how to build your own simple agent that can handle tasks using state management.

## Setup

### Installing Required Packages

Before we begin, let’s install the necessary Python libraries. Run the following command:

In [ ]:
%pip install requests jsonschema tenacity

These packages are used for:

- **requests:** Making HTTP requests to interact with models.
- **jsonschema:** Validating the structure of the agent's output.
- **tenacity:** Handling retries in case of errors when communicating with the model.

### Environment Setup

After installing the packages, we should verify that everything is set up correctly. Run the following code to check the installation:

In [ ]:
import requests
import jsonschema
import tenacity

## Understanding the Basics

### What is an Agent?

In simple terms, an agent is a program that can perform tasks autonomously based on a set of instructions. Think of an agent as a virtual assistant that can handle specific jobs for you. For example, in the context of data science or software development, an agent might process data, make decisions based on that data, and then carry out actions like sending requests or updating records.

Agents are often designed to work without constant human intervention. Once you give them the initial instructions, they can execute tasks on their own, making them very useful in automating repetitive or complex processes.

### State Management

State management is an essential concept when working with agents. The "state" of an agent refers to its current condition or the information it has at any given time. For instance, if an agent is working through a list of tasks, its state might include which tasks have been completed, which are in progress, and which are yet to be started.

Managing this state is crucial because it allows the agent to keep track of what it has done and what it needs to do next. Without proper state management, an agent might lose track of its progress, repeat tasks, or skip important steps. In this notebook, you'll learn how to manage an agent's state effectively, ensuring that it operates smoothly and efficiently.

## Step-by-Step Implementation

### 1. Building Blocks

In this section, we'll start by building the basic components that our agent will use. These building blocks will form the foundation of our agent, enabling it to keep track of time, store data, and interact with a model.

#### Datetime Function

The first building block we'll create is a simple function to get the current time. This is important because our agent might need to timestamp certain actions or events. Let's write a function that returns the current date and time in UTC format:


In [ ]:
from datetime import datetime, timezone


def get_current_utc_datetime():
    now_utc = datetime.now(timezone.utc)
    return now_utc.strftime("%Y-%m-%d %H:%M:%S.%f UTC")[:-3]


# Example usage:
print("Current UTC datetime:", get_current_utc_datetime())

This function gets the current time in Coordinated Universal Time (UTC) and formats it as a string, which can be easily used later in the agent's tasks.

### 2. Configuring a Simple Model
Now that we have the basic building blocks, we'll move on to configuring a model that our agent can use to perform its tasks. This model will process inputs (like a user request) and generate outputs (like a task list).

### Model Configuration
We'll start by setting up a simple configuration for the model. This configuration will include details like the model's endpoint, temperature, and other parameters. Let's create a function to handle this:

In [ ]:
def setup_ollama_model(
    model, temperature=0.0, top_p=1.0, top_k=0, repetition_penalty=1.0, stop=None
):
    return {
        "model_endpoint": "http://localhost:11434/api/generate",
        "model": model,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "repetition_penalty": repetition_penalty,
        "headers": {"Content-Type": "application/json"},
        "stop": stop,
    }


# Example configuration:
ollama_config = setup_ollama_model(model="llama3:instruct")
print("Model configuration:", ollama_config)

This function returns a dictionary with the model's configuration. You can adjust the parameters based on the specific model you're using or the task requirements.

### Preparing a Request
With the model configured, the next step is to prepare a request that the agent can send to the model. This request will include the user's input, the system's instructions, and any other necessary information. Let's write a function to prepare this request:

In [ ]:
from typing import Dict, Any

def prepare_payload(
    user_prompt: str,
    sys_prompt: str,
    stream: bool = False,
    config: Dict[str, Any] = ollama_config,
) -> Dict[str, Any]:
    return {
        "model": config.get("model"),
        "format": "json",
        "prompt": user_prompt,
        "system": sys_prompt,
        "stream": stream,
        "temperature": config.get("temperature", 0.0),
        "top_p": config.get("top_p", 1.0),
        "top_k": config.get("top_k", 0),
        "repetition_penalty": config.get("repetition_penalty", 1.0),
        "stop": config.get("stop"),
    }

In [ ]:
DEFAULT_SYS_PM_PROMPT = """
system

Environment: ipython  
Cutoff Knowledge Date: December 2023  

You are a **Task Management Agent**. Your job is to break down objectives into a clear and actionable task list. Each task must include a unique identifier, a name, a description, and a status. The status of each task must be one of the following: "pending", "in_progress", "completed", or "failed".

---

### Guidelines:

1. **Task List Structure:** Ensure that each task has the following properties:
   - `task_id`: A unique string identifier for the task.
   - `task_name`: A brief string that names the task.
   - `task_description`: A string that provides a detailed description of the task.
   - `status`: A string that represents the current status of the task. This must be one of the following: "pending", "in_progress", "completed", or "failed".

2. **No Additional Fields:** The task should only contain the properties listed above (`task_id`, `task_name`, `task_description`, and `status`). Do not include any additional properties.

3. **Task List Format:** The final output must be a JSON object with a single key `"tasks"`, which maps to an array of task objects. Each task object must follow the structure defined above.

---

### Output Format Example:

Your output should look like this:

{{
    "tasks": [
        {{
            "task_id": "task_001",
            "task_name": "Research and Choose Airplane Model",
            "task_description": "Select the most suitable airplane model for the project.",
            "status": "pending"
        }},
        {{
            "task_id": "task_002",
            "task_name": "Order Airplane Components",
            "task_description": "Order necessary airplane components according to the chosen model.",
            "status": "pending"
        }},
        {{
            "task_id": "task_003",
            "task_name": "Assemble Airplane Frame",
            "task_description": "Assemble the airplane frame according to the chosen model's specifications.",
            "status": "pending"
        }},
        {{
            "task_id": "task_004",
            "task_name": "Install Engine and Avionics",
            "task_description": "Install the engine and avionics system according to the chosen model's specifications.",
            "status": "pending"
        }},
        {{
            "task_id": "task_005",
            "task_name": "Add Controls and Instrumentation",
            "task_description": "Add controls and instrumentation according to the chosen model's specifications.",
            "status": "pending"
        }},
        {{
            "task_id": "task_006",
            "task_name": "Final Assembly and Testing",
            "task_description": "Perform final assembly and testing according to the chosen model's specifications.",
            "status": "pending"
        }}
    ]
}}

---

### Key Points:
- **Stick to the Structure:** Ensure each task has all the required properties and nothing else.
- **Maintain JSON Format:** The output must be valid JSON, with no extra fields or missing required fields.
"""

In [ ]:
# Example usage:
payload = prepare_payload(
    user_prompt="What tasks should I complete in order to make pasta?",
    sys_prompt=DEFAULT_SYS_PM_PROMPT.format(feedback=""),
)
print("Prepared payload:", payload)

This function creates a dictionary representing the request that will be sent to the model. The request includes the user's prompt, system instructions, and the configuration parameters for the model.

### 3. Interacting with the Model
Now that we have our model configured and a request prepared, let's demonstrate how to send this request to the model and process the response.

### Sending a Request
We'll start by writing a function to send the request to the model's endpoint and receive a response:

In [ ]:
import requests
import json


def request_model_generate_endpoint(
    payload: Dict[str, Any], config: Dict[str, Any] = ollama_config
) -> Dict[str, Any]:
    try:
        response = requests.post(
            config.get("model_endpoint"),
            headers=config.get("headers", {"Content-Type": "application/json"}),
            data=json.dumps(payload),
            timeout=30,
        )
        response.raise_for_status()

        if response.content.strip():
            return response.json()
        else:
            return {"error": "Empty response from model"}

    except requests.RequestException as e:
        raise Exception(f"Request failed: {e}")


# Example usage:
response = request_model_generate_endpoint(payload)
print("Model response:", response)

This function sends the prepared payload to the model's endpoint using the `requests` library. It then checks if the response is valid and returns the content. If there's an error in the request, it raises an exception with a descriptive message.

### Processing the Response
Finally, we'll write a function to process and understand the model's response. This might involve formatting the response or extracting specific information:

In [ ]:
def process_model_response(response_json: Dict[str, Any]) -> str:
    try:
        response_content = json.loads(response_json.get("response", "{}"))
        pretty_content = json.dumps(response_content, indent=4)
        
        return pretty_content
    except json.JSONDecodeError:
        return "Error processing the response"


# Example usage:
processed_response = process_model_response(response)
print("Processed response:", processed_response)

This function processes the model's response, formats it nicely, and returns it as a string. If there's an error in processing, it returns an error message.

## Putting It All Together

Now that we've built the core components of our agent, it's time to put everything together. In this section, we'll implement the agent, give it the ability to execute tasks, and demonstrate how to run it with a simple example.

### Implementing the Agent

#### Creating a Simple Agent Class

We'll start by creating a basic `Agent` class that utilizes the functions and concepts we've discussed so far. This class will manage the agent's state, interact with the model, and handle the responses.

#### Simple Task Execution
Next, let's implement a method within the `Agent` class that allows the agent to execute a simple task. This task could be anything from processing a user request to performing an operation based on model output.

In [ ]:
from typing import Dict, Any


class Agent:
    def __init__(self, state: Dict[str, Any], role: str, model_config: dict):
        """
        Initialize the Agent with a state, role, and model configuration.
        """
        self.state = state
        self.role = role
        self.model_config = model_config

    def update_state(self, key: str, value: Any):
        """
        Update the state of the agent. Warn if the key doesn't exist.
        """
        if key in self.state:
            self.state[key] = value
        else:
            print(f"Warning: Attempting to update a non-existing state key '{key}'.")

    def invoke_model(
        self, sys_prompt: str, user_prompt: str, update_state: bool = True
    ):
        """
        Prepare the payload, send the request to the model, and process the response.
        """
        # Prepare the payload
        payload = prepare_payload(user_prompt, sys_prompt, config=self.model_config)

        # Invoke the model and get the response
        response_json = request_model_generate_endpoint(
            payload, config=self.model_config
        )

        # Process the model's response
        response_content = process_model_response(response_json)

        # Optionally update the agent's state
        if update_state:
            self.update_state("response", response_content)

        # Return the processed response
        return response_content

    def work(self, user_request: str, feedback: str ="") -> str:
        """
        Execute a simple task based on the user's request.
        """
        # Define a simple system prompt
        sys_prompt = DEFAULT_SYS_PM_PROMPT.format(feedback=feedback)

        # Invoke the model with the user's request
        response = self.invoke_model(sys_prompt=sys_prompt, user_prompt=user_request)

        # Return the processed response
        return response

This class provides a basic framework for an agent, allowing it to maintain state, interact with a model, and process responses. The `invoke_model` method brings together the payload preparation, model invocation, and response processing steps.

In [ ]:
# Example usage:
# Initialize the agent with an empty state and a role
agent_state = {"response": ""}
agent_role = "Task Manager"
agent = Agent(state=agent_state, role=agent_role, model_config=ollama_config)

# Execute a task
user_input = "Create a migration plan."
response = agent.work(user_request=user_input)
print("Agent's response:", response)

The `work` method is a simple wrapper around the invoke_model method. It prepares the system prompt, sends the user's request to the model, and returns the processed response. This makes it easy to extend the agent's functionality as needed.

### Running the Agent

Finally, let's demonstrate how to run the agent with a simple example. We'll use the agent class we've just implemented to process a task based on user input.

In [ ]:
# Example: Running the agent
user_query = "Create a list of tasks in order to get a plane."

# Execute the task using the agent
result = agent.work(user_request=user_query)

# Output the result
print("Final Output:", result)

## Optional Advanced Section

While the basic agent we've implemented is functional, there are several ways to enhance its capabilities. In this section, we'll briefly cover two advanced topics: adding validation to the agent's output and extending the agent's functionality.

### Adding Validation

One important feature you can add to your agent is the ability to validate its output. This is particularly useful when the output must adhere to a specific format or schema, ensuring that the agent's responses are reliable and consistent.

Let's introduce a basic validation method using the `jsonschema` library, which we installed earlier. This method will check if the output from the model matches a predefined schema.


In [ ]:
from jsonschema import validate, ValidationError

# Example schema for validating the agent's output
output_schema = {
    "type": "object",
    "properties": {
        "tasks": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "task_id": {"type": "string"},
                    "task_name": {"type": "string"},
                    "task_description": {"type": "string"},
                    "status": {
                        "type": "string",
                        "enum": ["pending", "in_progress", "completed", "failed"],
                    },
                },
                "required": ["task_id", "task_name", "task_description", "status"],
            },
        }
    },
    "required": ["tasks"],
}

In [ ]:
def validate_agent_output(output: dict, schema: dict = output_schema) -> bool:
    """
    Validate the agent's output against the provided schema.
    """
    try:
        validate(instance=output, schema=schema)
        return True, ""
    except ValidationError as e:
        error_details = {
            "message": e.message,
        }
        return False, error_details


# Validate the example output
is_valid, message = validate_agent_output(json.loads(result))
print("Is the output valid?", is_valid)

In [ ]:
if not is_valid:
    # response_content = json.loads(message.get("message", "{}"))
    pretty_content = json.dumps(message, indent=4)
    print(pretty_content)
    # Execute the task using the agent

    result = agent.work(user_request=user_query, feedback=message)

    is_valid = validate_agent_output(json.loads(result))
    print("Is the output valid?", is_valid)

This function checks whether the output matches the expected structure. If the output is valid, it returns True; otherwise, it prints an error message and returns False. This validation step helps to catch errors early and ensure that your agent's output is always in the correct format.

### Extending the Agent

The agent we've created is quite simple, but there are many ways to extend its functionality. Here are a few ideas:

- **Multiple Roles:** You could extend the agent to handle multiple roles, each with different responsibilities. For example, an agent could manage tasks, provide recommendations, and monitor progress.

- **Dynamic State Management:** Instead of a static state, you could implement dynamic state management where the agent's state evolves based on complex conditions or inputs.

- **Error Handling:** Adding robust error handling will make your agent more reliable. You could catch specific exceptions and implement retry logic or fallback strategies.

- **Advanced Interactions:** Enhance the agent's interaction with the model by introducing more complex prompts, handling multi-turn conversations, or integrating additional APIs and services.

- **Custom Validation Rules:** Beyond schema validation, you can add custom validation rules tailored to specific tasks, ensuring that the agent's outputs meet your exact requirements.

These extensions can make your agent more powerful and adaptable, allowing it to handle more complex scenarios and provide more value.


### Conclusion and Next Steps

#### Recap

In this notebook, we've covered the essentials of building a simple agent in Python. You learned:

- **The basics of what an agent is** and how state management works.
- **How to set up the environment** and install the necessary packages.
- **How to build core components**, including a datetime function, data handling, and model configuration.
- **How to implement and run an agent**, with examples of task execution and response processing.

#### Further Learning

Now that you have a solid foundation, here are some next steps you can take to continue learning and improving your agent:

- **Explore more complex agents:** Look into building agents that can handle multiple tasks, manage dependencies, and operate in different environments.
- **Integrate with external APIs:** Extend your agent's capabilities by integrating it with external services, such as databases, cloud services, or messaging platforms.
- **Enhance error handling:** Implement more sophisticated error-handling techniques, such as retries, fallback mechanisms, and logging.
- **Experiment with different models:** Try using different models with your agent to see how they perform on various tasks. Explore fine-tuning models for specific tasks if needed.
- **Study advanced state management:** Learn about more complex state management techniques, such as finite state machines or event-driven architectures, to make your agent more robust.

We hope this notebook has provided you with a strong starting point for building and understanding agents in Python. Keep experimenting and exploring—there's much more to discover in the world of agents and automation!